# Web Scraping. MostWanted EU
## Selenium
### Santiago Fernández Seoane
## Importacion de librerias

In [1]:
#Instalar gecko
#from webdriver_manager.firefox import GeckoDriverManager
#GeckoDriverManager().install()
#!pip install requests lxml

#Abrir un navegador
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import json
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
import requests
from lxml import html
import base64



## Iniciamos el driver (en mi caso usando ruta absoluta, pero esta el comando para que lo detecte autmaticamente)

In [2]:
# Especifica la ruta al GeckoDriver
gecko_driver_path = 'C:/Users/constantin.madalin.i/mis-binarios/geckodriver.exe'


# Especifica la ruta al ejecutable de Firefox
firefox_binary_path = 'C:/Program Files/Mozilla Firefox/firefox.exe'

# Crea un objeto Options y establece la ruta al ejecutable de Firefox
options = Options()
options.binary_location = firefox_binary_path


# Crea un objeto Service con la ruta al GeckoDriver
service = Service(executable_path=gecko_driver_path, log_output='geckodriver.log')
driver = webdriver.Firefox(service=service, options=options)

In [3]:
#driver = webdriver.Firefox()

In [4]:
class WantedFields:
    def __init__(self):
        self.fields = {
            "wantedItem": "[class^='wantedItem']",
            "name": '//div[@class="field field--node-field-w-first-name field--name-field-w-first-name field--type-string field--label-hidden field__item"]',
            "crime": '//div[@class="field field--node-field-crime field--name-field-crime field--type-entity-reference field--label-inline clearfix"]/div[@class="field__items"]/div[@class="field__item"]',
            "gender": '//div[@class="field field--node-field-gender field--name-field-gender field--type-entity-reference field--label-inline clearfix"]/div[@class="field__item"]',
            "height": '//div[@class="field field--node-field-approximate-height field--name-field-approximate-height field--type-string field--label-inline clearfix"]/div[@class="field__item"]',
            "eyecolor": '//div[@class="field field--node-field-eye-colour field--name-field-eye-colour field--type-entity-reference field--label-inline clearfix"]/div[@class="field__item"]',
            "birthdate": '//div[@class="field field--node-field-date-of-birth field--name-field-date-of-birth field--type-datetime field--label-inline clearfix"]/div[@class="field__item"]',
            "nationality": '//div[@class="field field--node-field-nationality field--name-field-nationality field--type-entity-reference field--label-inline clearfix"]/div[@class="field__items"]/div[@class="field__item"]',
            "ethnic": '//div[@class="field field--node-field-ethnic-origin field--name-field-ethnic-origin field--type-entity-reference field--label-inline clearfix"]/div[@class="field__items"]/div[@class="field__item"]',
            "languages": '//div[@class="field field--node-field-languages field--name-field-languages field--type-entity-reference field--label-inline clearfix"]/div[@class="field__items"]/div[@class="field__item"]',
            "state": '//div[@class="field field--node-field-state-of-case field--name-field-state-of-case field--type-entity-reference field--label-inline clearfix"]/div[@class="field__item"]',
            "published": '//div[@class="field field--node-field-w-last-name field--name-field-w-last-name field--type-string field--label-inline clearfix"]/div[@class="field__item"]',
            "picture": '//div[@class="field field--node-field-picture field--name-field-picture field--type-image field--label-hidden field__items"]/div[@class="field__item show"]/img[1]',
            "arrested": '//div[@class="field field--node-field-arrested field--name-field-arrested field--type-datetime field--label-inline clearfix"]/div[@class="field__item"]'
        }

    def __getattr__(self, item):
        return self.fields.get(item, None)
    
oglink = 'https://eumostwanted.eu/'
wanted = WantedFields()

Abrimos la web y eliminamos el rectangulo de las cookies

In [5]:
driver.get('https://eumostwanted.eu/')
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'sort_close'))
)
element.click()

Funciones que utilizaremos repetidas veces

In [6]:
def waitUntil(by,path):
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((by, path)))

def get_element_text(by, value):
    try:
        element = driver.find_element(by, value)
        return element.text
    except NoSuchElementException:
        return None

def get_element_attribute(by, value, attribute):
    try:
        element = driver.find_element(by, value)
        return element.get_attribute(attribute) #get_atributte
    except NoSuchElementException:
        return None

def get_image_base64(by, value, attribute):
    try:
        image_url = get_element_attribute(by, value, attribute)
        image_response = requests.get(image_url)
        return base64.b64encode(image_response.content).decode('utf-8')
    except NoSuchElementException:
        return None

def make_screenshot():
    screenshot_path = os.path.join('buscados', f"{index}_{data['name']}.png")
    driver.save_screenshot(screenshot_path)

In [7]:
wanted_list = driver.find_element(By.CLASS_NAME, 'wanted_list')
wanted_items = wanted_list.find_elements(By.CSS_SELECTOR, wanted.wantedItem)
params = []
datafinal = {"buscados": []}

# Crear la carpeta 'buscados' si no existe
if not os.path.exists('buscados'):
    os.makedirs('buscados')

# Iteramos y guardamos el href de cada uno de los elementos de la lista de buscados para poder acceder correctamente
params = [wanted_suspect.get_dom_attribute('href') for wanted_suspect in wanted_items]

for index, url_suspect in enumerate(params):
    waitUntil(By.CSS_SELECTOR, wanted.wantedItem)

    if url_suspect != '':
        # Clickamos la celda de la lista de buscados
        driver.find_element(By.XPATH, f"//div[@href='{url_suspect}']").click()

        waitUntil(By.XPATH, wanted.picture)

        data = {
            "name": get_element_text(By.XPATH, wanted.name),
            "crime": get_element_text(By.XPATH, wanted.crime),
            "gender": get_element_text(By.XPATH, wanted.gender),
            "height": get_element_text(By.XPATH, wanted.height),
            "eyecolor": get_element_text(By.XPATH, wanted.eyecolor),
            "birthdate": get_element_text(By.XPATH, wanted.birthdate),
            "nationality": get_element_text(By.XPATH, wanted.nationality),
            "ethnic": get_element_text(By.XPATH, wanted.ethnic),
            "languages": get_element_text(By.XPATH, wanted.languages),
            "state": get_element_text(By.XPATH, wanted.state),
            "published": get_element_text(By.XPATH, wanted.published),
            "picture": get_image_base64(By.XPATH, wanted.picture, 'src')
        }
        datafinal["buscados"].append(data)
        # Hacemos la captura de pantalla
        make_screenshot()
        
        # Vamos atrás para volver a la lista de buscados
        driver.find_element(By.CLASS_NAME, "mobile").click()
    else:
        continue

# Guardar en un archivo json
with open('data.json', 'w') as outfile:
    json.dump(datafinal, outfile)


In [8]:
def create_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)

def get_wanted_params(wanted_items):
    params = []
    for wanted_suspect in wanted_items:
        param = wanted_suspect.get_dom_attribute('href')
        params.append(param)
    return params

def extract_suspect_data(wanted_suspect, index):
    driver.find_element(By.XPATH, f"//div[@href='{wanted_suspect}']").click()
    waitUntil(By.XPATH, wanted.picture)

    data = {
        "name": get_element_text(By.XPATH, wanted.name),
        "crime": get_element_text(By.XPATH, wanted.crime),
        "gender": get_element_text(By.XPATH, wanted.gender),
        "height": get_element_text(By.XPATH, wanted.height),
        "eyecolor": get_element_text(By.XPATH, wanted.eyecolor),
        "birthdate": get_element_text(By.XPATH, wanted.birthdate),
        "nationality": get_element_text(By.XPATH, wanted.nationality),
        "ethnic": get_element_text(By.XPATH, wanted.ethnic),
        "languages": get_element_text(By.XPATH, wanted.languages),
        "state": get_element_text(By.XPATH, wanted.state),
        "published": get_element_text(By.XPATH, wanted.published),
        "picture": get_image_base64(wanted.picture)
    }

    screenshot_path = os.path.join('buscados', f"{index}_{data['name']}.png")
    driver.save_screenshot(screenshot_path)
    driver.find_element(By.CLASS_NAME, "mobile").click()
    
    return data

def get_image_base64(image_xpath):
    image_url = get_element_attribute(By.XPATH, image_xpath, 'src')
    image_response = requests.get(image_url)
    return base64.b64encode(image_response.content).decode('utf-8')

def save_data_to_json(data, filename):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile)

# Main script
wanted_list = driver.find_element(By.CLASS_NAME, 'wanted_list')
wanted_items = wanted_list.find_elements(By.CSS_SELECTOR, wanted.wantedItem)

create_directory('buscados')
params = get_wanted_params(wanted_items)
datafinal = {"buscados": []}

for index, wanted_suspect in enumerate(params):
    waitUntil(By.CSS_SELECTOR, wanted.wantedItem)
    if wanted_suspect:
        data = extract_suspect_data(wanted_suspect, index)
        datafinal["buscados"].append(data)

save_data_to_json(datafinal, 'data.json')
